In [ ]:
# Импортируем библиотеки
import os
import pandas as pd
import seaborn as sn
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering


df = pd.read_csv(f'dataset/txns_data_raw/txns_ww0.csv', sep=';')
df.rename(columns = {'qw':'nn_qw'}, inplace = True)
df = df.dropna()
df.head()

In [ ]:
# Добавляем новую колокнку fee как сумма всех комиссий
df = df.assign(fee = df['client_fee'] + df['interchange_fee'] + df['payout_fee'] + df['paysys_fee'] + df['payin_fee'])
df['fee'].sum()

In [ ]:
df.dtypes

In [ ]:
# Определяем самый ранний денб покупки
df.sort_values(by='txn_minute')
df.head(20)

In [ ]:
df.sort_values(by='nn_qw', ascending=True).head()


In [ ]:
# Recency
day = '2022-12-26' # Начальная дата - самая ранняя
day = pd.to_datetime(day)
df['txn_minute'] =  pd.to_datetime(df['txn_minute']).dt.normalize()

recency = df.groupby(['nn_qw']).agg({'txn_minute': lambda x:(-1 * (day - x.max()).days)}) # Берем начальную и вычитаем из нее максимальную дату 
recency

In [ ]:
# Frequency
freq = df.groupby(['nn_qw'])[['nn_qw']].count()
freq

In [ ]:
# Monetary value

money = df.groupby(['nn_qw'])[['amount']].sum()
money

In [ ]:
recency.columns=['Recency']
freq.columns=['Frequency']
money.columns=['Monetary']
RFM = pd.concat([recency, freq, money], axis=1)
RFM

In [ ]:
# Машинное обучение
scaler = StandardScaler()
scaled = scaler.fit_transform(RFM)

In [ ]:
# Выбираем количество кластеров
inertia = []
for i in np.arange(1, 11):
    kmeans = KMeans(n_clusters=i, init = 'k-means++')
    kmeans.fit(scaled)
    inertia.append(kmeans.inertia_)

plt.plot(inertia, marker='o')

In [ ]:
# Из графика видно, что нам нужно 3 Кластера
kmeans = KMeans(n_clusters=3)
kmeans.fit(scaled)
RFM['Clusters'] = (kmeans.labels_ + 1)
RFM

In [ ]:
group = RFM.groupby(['Clusters'])['Recency', 'Frequency', 'Monetary'].mean()
group

In [ ]:
def distribution(row):
    if row['Clusters'] == 1:
        return '222' # Спящие редкие со средним чеком
    elif row['Clusters'] == 2:
        return '311' # Недавние разовые покупки с низким чеком
    else:
        return '233' # Спящие частые с высоким чеком

In [ ]:
RFM['Conditions'] = RFM.apply(distribution, axis=1)
results = RFM['Clusters'].value_counts()
results.plot(kind='barh', color=['green', 'blue', 'red'])

In [ ]:
RFM.groupby('Clusters').size()

In [ ]:
# Записываем файл
# RFM.to_csv('RFM_0.csv')

In [ ]:
# import plotly.express as px
# px.scatter_3d(RFM['Recency'].to_list(), RFM['Frequency'].to_list(), RFM['Monetary'].to_list())
